In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split


In [ ]:
# read table

dataPayment = '../Datasets/Employee_Payroll.csv'
payroll = pd.read_csv(dataPayment)

# set default value to 0 for NaN numerical data
numeric_cols = payroll.select_dtypes(include=[np.number]).columns
payroll[numeric_cols] = payroll[numeric_cols].clip(lower=0)


payroll.fillna(0, inplace=True)
payroll['Office'] = payroll['Office'].astype(int)

# define column for 1/4 year discretization
payroll['Fiscal Quarter'] = payroll['Fiscal Quarter']*0.25 - 0.25
payroll['Fiscal Period'] = payroll['Fiscal Year'] + payroll['Fiscal Quarter']

pd.to_datetime(payroll['Original Hire Date'])

# parse hire date to get hire year
payroll['Original Hire Date'] = payroll['Original Hire Date'].str.split('/').str[2]
payroll['Original Hire Date'] = payroll['Original Hire Date'].astype(int)

_work_year = payroll["Fiscal Period"] - payroll['Original Hire Date']

In [ ]:
# specify table column included

payroll = payroll[['Fiscal Year', 'Fiscal Period', 'Job Code', 'Job Title', 'Base Pay', 'Position ID', 'Employee Identifier']]

# add Working Year Column
payroll["Working Year"] = _work_year

payroll.head(20)

In [ ]:
X = payroll[['Job Code','Working Year']]
Y = payroll['Base Pay'].values
X.head(10)

In [ ]:
#view target values
Y[0:5]

In [ ]:
x_train, x_test,y_train,y_test = train_test_split(X,Y,test_size =0.2)
# print the data
x_train

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
# Create KNN classifier
knn = KNeighborsRegressor(n_neighbors = 3)
# Fit the classifier to the data
knn.fit(x_train,y_train)

In [ ]:
#show first 5 model predictions on the test data
y_pred = knn.predict(x_test)

In [ ]:
#check accuracy of our model on the test data
knn.score(x_test, y_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

In [ ]:
# YANG INI JANGAN DIJALANKAN DULU YA GESSS :'

arr = []

try:
    with open('./cache/ids.txt', "r") as f:
        for _id in f:
            arr.append(int(_id))
except:
    if not os.path.exists('./cache'):
        os.mkdir('./cache')
        
    _index = payroll['Job Code'].unique()
    for _id in _index:
        counts = len(payroll.groupby(['Job Code']).get_group(_id))
        if counts > 1000:
            arr.append(str(_id))
    with open('cache/ids.txt', 'w') as f:
        for _id in arr:
            f.write('%s\n' % _id) 
finally:
   f.close()

# :return : <List> arr : list of unique job id

In [ ]:
# Group Job Code 1172 training dataset <<< PAKAI INI NGGIH

group_1172 = payroll.groupby(['Job Code']).get_group(1172)

_sorted = group_1172.sort_values('Employee Identifier')
fiscal = pd.pivot_table(_sorted, values='Base Pay', index=['Fiscal Period'], columns='Employee Identifier')

fiscal = fiscal.diff()

In [ ]:
# cleaned display table

fiscal.fillna(0, inplace=True)
fiscal = fiscal.drop(2016.00)

fiscal

In [ ]:


# neg_cols = (payroll[numeric_cols] < 0).any()

# if neg_cols.any():
#     print("Terdapat nilai negatif pada kolom: ", end="")
#     print(", ".join(neg_cols[neg_cols == True].index))
# else:
#     print("Tidak terdapat nilai negatif pada semua kolom numerik.")

In [ ]:
# SUDAH AMAN 👍👍

# null_cols = payroll.isnull().any()

# if null_cols.any():
#     print("Terdapat nilai NaN pada kolom: ", end="")
#     print(", ".join(null_cols[null_cols == True].index))
# else:
#     print("Tidak terdapat nilai NaN pada semua kolom.")

In [ ]:
# query job code indexing

for val in arr:
    new_df = payroll[payroll['Job Code'] == val]
    print(f"DataFrame for job code {val}:")
    display(new_df)